In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
aaTable = {"Ala":"A",
           "Arg": "R",
           "Asn": "N",
           "Asp": "D",
           "Cys": "C",
           "Gln": "Q",
           "Glu": "E",
           "Gly": "G",
           "His": "H",
           "Ile": "I",
           "Leu": "L",
           "Lys": "K",
           "Met": "M",
           "Phe": "F",
           "Pro": "P",
           "Ser": "S",
           "Thr": "T",
           "Trp": "W",
           "Tyr": "Y",
           "Val": "V"}

In [ ]:
import pickle

In [ ]:
npid2seq = pickle.load(open("/data/projects/processBio/ncbi/npid2seq.pkl","rb"))
# npid2seq = pickle.load(open("/ssdata/projects"))

# gnomAD

In [ ]:
df = pd.read_csv("/data/projects/processBio/gnomad/gnomad.exomes.r2.1.1.sites.vcf.vep",
                 header=93,delimiter="\t")

In [ ]:
df

Originally 103,321,415 variants

In [ ]:
df = df[df.Consequence.str.contains("missense")]

In [ ]:
df

1,652,025 missense variants

In [ ]:
df = df.assign(info=df.Extra.apply(lambda s: dict([l.split("=") for l in s.split(";")])))

In [ ]:
df.Feature.unique().shape

In [ ]:
from ensembl_rest import sequence_id, lookup, symbol_lookup, HTTPError
def getSequenceFromTranscript(transcriptID):
    return sequence_id(lookup(transcriptID,
                              params={"expand":True})["Translation"]["id"])["seq"]

In [ ]:
transcripts = df.Feature.unique()

In [ ]:
gnomadTranscriptToSeq = {}
for t in tqdm(transcripts):
    gnomadTranscriptToSeq[t] = getSequenceFromTranscript(t)

In [ ]:
import pickle

In [ ]:
pickle.dump(gnomadTranscriptToSeq, open("/data/projects/processBio/gnomad/transcriptToSeq.pkl","wb"))

In [ ]:
gnomadTranscriptToSeq = pickle.load(open("/data/projects/processBio/gnomad/transcriptToSeq.pkl","rb"))

In [ ]:
def makeGnomADSeq(row):
    og,var = row.Amino_acids[0], row.Amino_acids[-1]
    loc = int(row["Protein_position"]) - 1
    refSeq= gnomadTranscriptToSeq[row["Feature"]]
    if len(refSeq) <= loc:
        print("invalid seq len")
        return ""
    if refSeq[loc] == og:
        return refSeq[:loc] + var + refSeq[loc+1:]
    print("invalid reference AA")
    return ""

In [ ]:
df = df.assign(seq=df.apply(lambda row: makeGnomADSeq(row),axis=1))

In [ ]:
df[df.seq != ""]

In [ ]:
df.to_pickle("/data/projects/processBio/gnomad/df.reprocessed.pd.pkl")

In [ ]:
df

# HGMD

In [ ]:
hgmd = pd.read_csv("/data/projects/processBio/hgmd/HGMD_PRO_2019_1_hg19.vcf",delimiter="\t",header=14)

In [ ]:
hgmd = hgmd.assign(INFO = hgmd.INFO.apply(lambda s: dict([l.split("=") for l in s.split(";")])))

In [ ]:
hgmd

Originally 229,161 HGMD variants

In [ ]:
hgmd = hgmd[hgmd.INFO.apply(lambda i: i["CLASS"] == "DM")]

In [ ]:
hgmd

180,092 variants with class DM

In [ ]:
from Bio import Entrez
Entrez.email = 'zeiberg.d2@northeastern.edu'

def getSequenceFromNPID(npid):
    "Return the protein sequence from "
    handle = Entrez.efetch(db="protein",id=npid, rettype="fasta", retmode="text")
    lines = handle.readlines()
    lines = [l.strip() for l in lines]
    return "".join(lines[1:])

In [ ]:
def getProt(i):
    if "PROT" in i:
        return i["PROT"][:i["PROT"].find(":")]
    return ""

def hasProt(i):
    if "PROT" in i:
        return 1
    return 0

In [ ]:
def getHGMDVariantInfo(i):
    if "PROT" not in i:
        return (np.nan, np.nan)
    npid, variant = i["PROT"].split(":")
    variant = variant[2:]
    og,loc,var = variant[0], variant[1:-1], variant[-1]
    if og not in aaTable.values() or var not in aaTable.values():
        return (np.nan, np.nan)
    try:
        loc = int(loc) - 1
    except ValueError:
        return (np.nan, np.nan)
    return (npid, (og,loc,var))

In [ ]:
hgmdVariantInfo = hgmd.INFO.apply(lambda i: getHGMDVariantInfo(i))

In [ ]:
hgmd = hgmd.assign(variantInfo = hgmdVariantInfo[hgmdVariantInfo.apply(lambda t: not pd.isna(t[0]))])

In [ ]:
hgmd

In [ ]:
hgmd[~hgmd.variantInfo.isna()]

In [ ]:
def getHGMDNPIDs(info):
    if not pd.isna(info):
        npid,_ = info
        if npid not in npid2seq:
            npid2seq[npid] = getSequenceFromNPID(npid)
    

In [ ]:
for info in tqdm(hgmd.variantInfo):
    getHGMDNPIDs(info)

In [ ]:
def makeHGMDSeq(info):
    if pd.isna(info):
        return ""
    npid, (og,loc,var) = info
    refSeq = npid2seq[npid]
    if len(refSeq) < loc:
        return ""
    if refSeq[loc] != og:
        return ""
    return refSeq[:loc] + var + refSeq[loc+1:]

In [ ]:
hgmdSeqs = []
for info in tqdm(hgmd.variantInfo):
    hgmdSeqs.append(makeHGMDSeq(info))

In [ ]:
hgmd = hgmd.assign(seq=hgmdSeqs)

In [ ]:
hgmd[hgmd.seq != ""]

# Clinvar

In [ ]:
summary = pd.read_csv("/data/projects/processBio/clinvar/clinvar/variant_summary.txt",delimiter="\t")


In [ ]:
summary = summary[(summary.Type == "single nucleotide variant") & (summary.ReviewStatus.isin(["criteria provided, single submitter",
              "criteria provided, multiple submitters, no conflicts",
              "reviewed by expert panel",
              "practice guideline",
              ])) & (summary.Assembly == "GRCh38")]

In [ ]:
summary = summary.assign(nmid=summary.Name.apply(lambda n: n[:n.find("(")]))

In [ ]:
summary

767,700 single nucleotide variants with reference genome GRCh38 with one-star review status

In [ ]:
g2rs = pd.read_csv("/data/projects/processBio/ncbi/gene2refseq",delimiter="\t")

In [ ]:
g2rs = g2rs[(g2rs.status == "REVIEWED") 
            & (g2rs.assembly == "Reference GRCh38.p13 Primary Assembly")
            & (g2rs["protein_accession.version"] != "-")]

In [ ]:
nmid2npid = {}
for nmid in tqdm(summary.nmid.unique()):
    nmid2npid[nmid] = g2rs[g2rs["RNA_nucleotide_accession.version"] == nmid]["protein_accession.version"].unique()

In [ ]:
summary = summary.assign(npid=summary.nmid.apply(lambda n: nmid2npid[n]))

In [ ]:

for npid in tqdm(set([s[0] for s in summary.npid.values if len(s)])):
    if npid not in npid2seq:
        try:
            npid2seq[npid] = getSequenceFromNPID(npid)
        except Entrez.HTTPError:
            raise

In [ ]:
pickle.dump(npid2seq,open("/data/projects/processBio/ncbi/npid2seq.pkl","wb"))

In [ ]:
summary

In [ ]:
summary = summary.assign(npid=summary[summary.npid.str.len() != 0].npid.apply(lambda n: n[0] if len(n) else np.nan))

In [ ]:
summary[~summary.npid.isna()]

623,532 variants found a matching protein id

In [ ]:
summary[~summary.npid.isna()].ClinicalSignificance.value_counts()

In [ ]:
clinvar = summary[(~summary.npid.isna()) & (summary.ClinicalSignificance.isin(["Likely benign",
                                                                     "Benign", 
                                                                     "Pathogenic",
                                                                     "Likely pathogenic",
                                                                     "Benign/Likely benign",
                                                                     "Pathogenic/Likely pathogenic"]))]

In [ ]:
clinvar.loc[33]

328,243 variants have matching protein and valid status

In [ ]:
def extractVariantFromClinvar(n):
    if n.rfind("(p.") != -1:
        variant = n[n.rfind("(p.") + 3 : -1]
#         print(variant)
        og,loc,var = variant[:3], variant[3:-3], variant[-3:]
        try:
            loc = int(loc) -1
        except ValueError:
            return np.nan, np.nan,np.nan
        if variant[-1] == "=":
#             print("variant is =")
            return np.nan, np.nan,np.nan
        return og,loc,var
    return np.nan, np.nan, np.nan

In [ ]:
clinvar = clinvar.assign(variantInfo=clinvar.Name.apply(lambda n: extractVariantFromClinvar(n)))

In [ ]:
# clinvar = clinvar[clinvar.variantInfo.apply(lambda t: not pd.isna(t[0]))]
clinvar[clinvar.variantInfo.apply(lambda t: pd.isna(t[0]))].RCVaccession

In [ ]:
clinvar

73,371 variants have matching protein, valid status, and have alternate residue != "="

In [ ]:
clinvar.ClinicalSignificance.value_counts()

In [ ]:
clinvar

In [ ]:
def makeClinvarSeq(row):
    npid = row["npid"]
    refSeq = npid2seq[npid]
    og,loc,var = row["variantInfo"]
    if og not in aaTable or var not in aaTable:
        if var != "Ter":
            print('invalid parsing of variant... ', og, loc, var)
        return ""
    og = aaTable[og]
    var = aaTable[var]
    if len(refSeq) <= loc:
        print('invalid seq len')
        return ""
    if refSeq[loc] != og:
        print('invalid ref AA')
        return ""
    return refSeq[:loc] + var + refSeq[loc+1:]

In [ ]:
clinvar = clinvar.assign(seq = clinvar.apply(lambda row: makeClinvarSeq(row),axis=1))

In [ ]:
clinvar[clinvar.seq != ""]

Successfully retreived 55,087 variants

# Combine Labeled Data

In [ ]:
hgmd = hgmd.assign(ClinSigSimple=1)
hgmd = hgmd.assign(db="hgmd")

In [ ]:
clinvar = clinvar.assign(db="clinvar")

In [ ]:
hgmd = hgmd.assign(npid = hgmd.variantInfo.apply(lambda i: i[0] if not pd.isna(i) else np.nan))

In [ ]:
labeledDF = pd.concat((hgmd[hgmd.seq != ""][["ClinSigSimple", "seq", "npid", "db","variantInfo"]],
  clinvar[clinvar.seq != ""][["npid", "seq", "ClinSigSimple", "db","variantInfo"]]))

In [ ]:
labeledDF

In [ ]:
labeledDF.npid.unique().shape

In [ ]:
labeledDF.ClinSigSimple.value_counts()

In [ ]:
labeledDF.groupby("npid").count().seq.mean()

In [ ]:
(labeledDF.groupby("npid").count() > 100).sum()

In [ ]:
import torch
import esm
import os
import torch.nn as nn
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1,2,3'


model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()

In [ ]:
model = model.to("cuda:2")

In [ ]:
from tqdm.notebook import trange

In [ ]:
def embedSeq(Data):
    BATCHSIZE=1
    representations = []
    for start in trange(0,len(Data),BATCHSIZE):
        batch_labels, batch_strs, batch_tokens = batch_converter(Data[start : start + BATCHSIZE])
        # Extract per-residue representations (on CPU)
        with torch.no_grad():
            results = model(batch_tokens.to("cuda:2"), repr_layers=[33], return_contacts=True)
        token_representations = results["representations"][33].cpu()
        del results, batch_labels, batch_strs, batch_tokens
        representations.append(token_representations[0,1:-1].cpu().numpy())
        del token_representations
    return representations

In [ ]:
def prepSeq(row):
    if type(row["variantInfo"][1]) == tuple:
        loc= row["variantInfo"][1][1]
    else:
        loc = row["variantInfo"][1]
    seq = row["seq"]
    return seq[max(0,loc-510) : min(len(seq), loc + 510)]

In [ ]:
labeledDF = labeledDF.assign(preppedSeq = labeledDF.apply(lambda row: prepSeq(row),axis=1))

In [ ]:
df.head()

In [ ]:
def prepGnomad(row):
    loc = int(row["Protein_position"])-1
    seq = row["seq"]
    return seq[max(0,loc-510) : min(len(seq), loc + 510)]

In [ ]:
df= df.assign(preppedSeq=df.apply(lambda row: prepGnomad(row),axis=1))

In [ ]:
from tqdm.notebook import trange

In [ ]:
GnomadData = list(df.preppedSeq.items())

In [ ]:
gnomadEmbeddings = embedSeq(GnomadData)

In [ ]:
df = df.assign(embedding=gnomadEmbeddings)

In [ ]:
df.to_pickle('/data/projects/processBio/gnomadReprocessed.pkl')


In [ ]:
Data = list(labeledDF.preppedSeq.items())

In [ ]:
embeddings = embedSeq(Data)

In [ ]:
embeddings

In [ ]:
labeledDF = labeledDF.assign(embedding=embeddings)

In [ ]:
labeledDF

In [ ]:
import pickle

In [ ]:
labeledDF = labeledDF.assign(vec=labeledDF.embedding.apply(lambda e: e.mean(0)))

In [ ]:
labeledDF.to_pickle("/data/projects/processBio/labeledDFReprocessed.pkl")

In [ ]:
labeledDF.ClinSigSimple.value_counts()

In [ ]:
labeledDF.groupby("npid").ClinSigSimple.value_counts().describe()

In [ ]:
ls /data/projects/processBio/